<a href="https://colab.research.google.com/github/rojinadeuja/NLP-Model-Implementations/blob/master/GloVe_using_Gensim_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
GLOVE_DIR = '/content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt'
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/IMDB.csv'

df = pd.read_csv(DATA_DIR)
df.replace(['positive', 'negative'], [1, 0], inplace=True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Data pre-processing

data = list(df.review)
labels = list(df.sentiment)

tokenizer = Tokenizer(num_words=10000) # Only 10000 most met words
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

In [8]:
data = pad_sequences(sequences, maxlen=100) # Only 100 words in one review
labels = np.array(labels)

train_data = data[:30000]
train_labels = labels[:30000]

validation_data = data[30000:40000]
validation_labels = labels[30000:40000]

test_data = data[40000:]
test_labels = labels[40000:]

In [11]:
embedding_index = {}
with open(GLOVE_DIR) as file: # Preparing glove
    for line in file:
        values = line.split()
        key = values[0]
        value = values[1:]
        embedding_index[key] = np.array(value, dtype='float32')
        
        
embedding_matrix = np.zeros((10000, 100)) # 10000 - num_words, 100 - length of sequence
for word, i in tokenizer.word_index.items():
    if i < 10000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector